In [1]:
from __future__ import print_function
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [6]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [7]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)



Using real-time data augmentation.
Epoch 1/20
1562/1562 [==============================] - 275s - loss: 1.8719 - acc: 0.3115 - val_loss: 1.5986 - val_acc: 0.4206
Epoch 2/20
1562/1562 [==============================] - 274s - loss: 1.5888 - acc: 0.4210 - val_loss: 1.4219 - val_acc: 0.4848
Epoch 3/20
1562/1562 [==============================] - 275s - loss: 1.4557 - acc: 0.4712 - val_loss: 1.3019 - val_acc: 0.5335
Epoch 4/20
1562/1562 [==============================] - 272s - loss: 1.3635 - acc: 0.5083 - val_loss: 1.1787 - val_acc: 0.5823
Epoch 5/20
1562/1562 [==============================] - 275s - loss: 1.2905 - acc: 0.5399 - val_loss: 1.1207 - val_acc: 0.6000
Epoch 6/20
1562/1562 [==============================] - 289s - loss: 1.2232 - acc: 0.5640 - val_loss: 1.0740 - val_acc: 0.6209
Epoch 7/20
1562/1562 [==============================] - 278s - loss: 1.1738 - acc: 0.5848 - val_loss: 1.0039 - val_acc: 0.6529
Epoch 8/20
1562/1562 [==============================] - 299s - loss: 1.1286 

In [8]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'

Saved trained model at C:\Users\Derek\Anaconda3\Scripts\saved_models\keras_cifar10_trained_model.h5 


In [9]:
keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

In [10]:
# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

Model Accuracy = 0.72


In [11]:
for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Actual Label = cat vs. Predicted Label = frog
Actual Label = ship vs. Predicted Label = airplane
Actual Label = ship vs. Predicted Label = airplane
Actual Label = airplane vs. Predicted Label = horse
Actual Label = frog vs. Predicted Label = deer
Actual Label = frog vs. Predicted Label = dog
Actual Label = automobile vs. Predicted Label = frog
Actual Label = frog vs. Predicted Label = cat
Actual Label = cat vs. Predicted Label = automobile
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = airplane vs. Predicted Label = cat
Actual Label = truck vs. Predicted Label = frog
Actual Label = dog vs. Predicted Label = cat
Actual Label = horse vs. Predicted Label = horse
Actual Label = truck vs. Predicted Label = deer
Actual Label = ship vs. Predicted Label = airplane
Actual Label = dog vs. Predicted Label = bird
Actual Label = horse vs. Predicted Label = bird
Actual Label = ship vs. Predicted Label = truck
Actual Label = frog vs. Predicted Label = horse
Actual Label = h